# Crafting Recommedation System with PySpark

This Notebook uses the [movielens data set](https://grouplens.org/datasets/movielens/)

Collaborative Filtering : https://developers.google.com/machine-learning/recommendation/collaborative/basics

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
#!ls ./ml-latest-small/

In [9]:
!hdfs dfs -ls /demo/recommendation

Found 1 items
-rw-r--r--   1 root supergroup      14373 2020-11-26 07:48 /demo/recommendation/movielens_ratings.csv


In [10]:
data = spark.read.csv('/demo/recommendation/movielens_ratings.csv',inferSchema=True,header=True)

In [11]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [12]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [13]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [14]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [15]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [18]:
predictions.show(10,50)

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   2.0|    25|-0.98915493|
|     85|   1.0|    15| 0.34672984|
|     85|   4.0|     7|  4.4518023|
|     65|   2.0|     5|   1.967547|
|     65|   1.0|    19|  1.5886496|
|     65|   2.0|    15|  0.9058608|
|     65|   1.0|    24| -1.3335713|
|     65|   1.0|     2|  0.9489412|
|     53|   1.0|    12|   3.206553|
|     53|   1.0|     6| 0.21347225|
+-------+------+------+-----------+
only showing top 10 rows



In [19]:
predictions = predictions.na.drop()

In [20]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 2.043650081699683


# Prediction pour un utilisateur

In [21]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [22]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|     12|    11|
|     13|    11|
|     22|    11|
|     25|    11|
|     36|    11|
|     38|    11|
|     39|    11|
|     48|    11|
|     62|    11|
|     75|    11|
|     76|    11|
|     77|    11|
|     88|    11|
|     90|    11|
+-------+------+



In [23]:
reccomendations = model.transform(single_user)

In [24]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     62|    11|  4.5685115|
|     75|    11|  3.4750867|
|     36|    11|  3.2745628|
|     48|    11| 0.92061293|
|      6|    11|  0.8001625|
|     25|    11| 0.78817904|
|     39|    11|  0.6231711|
|     22|    11| -0.1868838|
|     76|    11| -0.2185092|
|     90|    11|-0.47727418|
|     13|    11| -0.5353743|
|     88|    11| -0.6190188|
|     12|    11| -2.3134363|
|     77|    11| -2.9219615|
|     38|    11| -5.3621693|
+-------+------+-----------+



# Exercice

- Améliorer le score
- Regarder si les prédictions sont cohérentes en exploitant le fichier movies.csv.
- Faite une jointure

In [7]:
!head ./ml-latest-small/movies.csv

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action
